# Stack Overflow Annual Developer Survey 2020
## Analyse Minorities Answers 


This is a project 1 from Udacity Data Scients Nanodegree, the gol is to find out about this questions:


* How minorities breaks IT fields 
* Company values, size and Job satisfaction 
* On-boarding Process, Job Hunt Factors

Data Source: [https://insights.stackoverflow.com/survey/]


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline



In [2]:
# Import data frames 
df = pd.read_csv('./developer_survey_2020/survey_results_public.csv')
df.head()

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",ASP.NET Core,ASP.NET;ASP.NET Core,Just as welcome now as I felt last year,50.0,36,27
1,2,I am a developer by profession,No,NaN,19,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat more welcome now than last year,NaN,7,4
2,3,I code primarily as a hobby,Yes,NaN,15,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,NaN,NaN,Somewhat more welcome now than last year,NaN,4,NaN
3,4,I am a developer by profession,Yes,25.0,18,NaN,NaN,NaN,Albania,Albanian lek,...,NaN,NaN,No,"Computer science, computer engineering, or sof...",NaN,NaN,Somewhat less welcome now than last year,40.0,7,4
4,5,"I used to be a developer by profession, but no...",Yes,31.0,16,NaN,NaN,NaN,United States,NaN,...,Easy,Too short,No,"Computer science, computer engineering, or sof...",Django;Ruby on Rails,Ruby on Rails,Just as welcome now as I felt last year,NaN,15,8


In [ ]:
schema = pd.read_csv('./developer_survey_2020/survey_results_schema.csv')
schema

In [4]:
# Search by columns name  
schema[schema["Column"].str.contains("Ethni")]

NameError: name 'schema' is not defined

In [ ]:
# Quantity groups by Country, Ethinicity  and Gender 
df[['Country', 'Ethnicity', 'Gender']].value_counts().head(50)

#### Check percent nulls  

In [ ]:
np.sum(df['Country'].isnull())/df.shape[0]

In [ ]:
np.sum(df['Ethnicity'].isnull())/df.shape[0]

In [ ]:
np.sum(df['Gender'].isnull())/df.shape[0]

In [ ]:
# Remove nulls in the mains colununs 
df_dropnan = df.dropna(subset=['Ethnicity','Gender'])

In [ ]:
# Quantity after excluded null values 
df_dropnan[['Ethnicity','Gender']].value_counts()

## % of White  Man respondents 

In [ ]:
df_dropnan[(df_dropnan['Ethnicity'].str.contains("White")  &  df_dropnan['Gender'].str.contains("Man"))].shape[0]/df_dropnan.shape[0]

## White Man is 62,7 % of all respondents

In [ ]:
# Not White Man respondents 
df_minority = df_dropnan[~(df_dropnan['Ethnicity'].str.contains("White")  &  df_dropnan['Gender'].str.contains("Man"))]

In [ ]:
df_minority.groupby(['Ethnicity', 'Gender']).Respondent.count().sort_values(ascending=False).head(30)

Now we can see a diferente mojority like South Asian people in India, them we will remove the major male ethnicity in each country. Becouse  we want to listening every woman then we will not exclue they in our study idempendent her ethinicity. 

In [ ]:
# Top Ethnicity in each country 
df_country_top_ethnicity = df_dropnan.groupby(['Country', 'Ethnicity'], as_index = False)['Respondent'].count()
df_country_top_ethnicity = df_country_top_ethnicity.sort_values(['Respondent'], ascending = False).groupby(['Country'], as_index=False).head(1)
df_country_top_ethnicity.head(20)

In [ ]:
# Gender distribution 
df_dropnan['Gender'].value_counts()#/df_dropnan.shape[0]

In [ ]:
# Exclude Gender categories with names start "man"
df_dropnan[df_dropnan['Gender'].str.fullmatch("^man" , flags = 2)]

In [ ]:
# DataFrame withought top ethnicity mans in each country

#Merge df_dropnan and df_country_top_ethnicity
majority = pd.merge(df_dropnan, df_country_top_ethnicity, how = "left", on = ['Country', 'Ethnicity'])

# Gender start with man  and has in top ethnicity in country 
majority = majority[ majority['Gender'].str.fullmatch("^man" , flags = 2) & (~ pd.isna(majority['Respondent_y'])) ]

In [ ]:
majority[['Country', 'Ethnicity', 'Gender']].value_counts().head(50)

In [ ]:
# Minority is not in majority dataframe
diversity = df_dropnan[~df_dropnan['Respondent'].isin(majority['Respondent_x'])]
diversity [['Respondent', 'Country', 'Ethnicity', 'Gender']].head(50)

Now we can see diversity people in the diversity dataframe. 
To recap, now we will see the answers from the not man and dominant ethnicity in country in other words we selected every Womans and not binary gender and mans not dominant ethnicity

### Transform Gender groups, for more easy analysis we will group in 3 categories 

* Man 
* Woman 
* Non-binary

In [ ]:
# Gender distribution in diversity dataframe 
diversity["Gender"].value_counts()/diversity.shape[0]

In [ ]:
sns.countplot( y="Gender", data=diversity)

Mans continue in larger number even in diversity group, maybe we will use weights to make better distribution. 

### Select Columns  to investigate 

* JobFactors

Imagine that you are deciding between two job offers with the same compensation, benefits, and location. Of the following factors, which 3 are MOST important to you?  

* NEWJobHuntResearch

 When job searching, how do you learn more about a company? Select all that apply.  
 
 
* JobSat
 
   How satisfied are you with your current job? (If you work multiple jobs, answer for the one you spend the most hours on.) 

In [ ]:
#Find questions to diversity people metters to select companys to work. 
with pd.option_context("max_colwidth", None):
    print(schema[ schema['QuestionText'].str.contains("Job", case=False)])

In [ ]:
with pd.option_context("max_rows", None):
    print(schema)